In [1]:
import pandas as pd
import geopandas as gpd
import duckdb

In [50]:
con = duckdb.connect("seattle.db")

In [ ]:
con.install_extension("spatial")
con.

In [29]:
#import csv
df = pd.read_csv('../data/csv/socio-economic-factors_20220905.csv')
#select only geo_fip, population_density and Median Household Income (In 2019 Inflation Adjusted Dollars)
df = df[['Geo_FIPS', 'Population Density (Per Sq. Mile)', 'Median Household Income (In 2019 Inflation Adjusted Dollars)']]
#rename columns to GEOID10, pop_density and median_income
df = df.rename(columns={'Geo_FIPS': 'GEOID10', 'Population Density (Per Sq. Mile)': 'pop_density', 'Median Household Income (In 2019 Inflation Adjusted Dollars)': 'median_income'})
# drop null values
df = df.dropna()
#convert GEOID10 to int
df['GEOID10'] = df['GEOID10'].astype(int)
df

,GEOID10,pop_density,median_income
0,530330001001,3708.992,206400.0
1,530330001002,14342.320,50714.0
2,530330001003,10101.800,51397.0
3,530330001004,51934.570,42150.0
4,530330001005,15089.690,47292.0
...,...,...,...
475,530330121001,3758.872,140400.0
476,530330121002,4571.933,117727.0
479,530330260011,5680.552,104469.0
480,530330264004,2000.136,62188.0


In [28]:
#load shapefile
bg = gpd.read_file('../data/shapefiles/Census_Block_Groups_2010/Census_Block_Groups_2010.shp')
#select columns GEOID, geometry
bg = bg[['GEOID10', 'geometry']]
#convert GEOID10 to int
bg['GEOID10'] = bg['GEOID10'].astype(int)
bg

,GEOID10,geometry
0,530330001001,"POLYGON ((-122.26840 47.72641, -122.26719 47.7..."
1,530330001002,"POLYGON ((-122.28970 47.73193, -122.28965 47.7..."
2,530330001003,"POLYGON ((-122.28633 47.72093, -122.28629 47.7..."
3,530330001004,"POLYGON ((-122.29237 47.73194, -122.29231 47.7..."
4,530330001005,"POLYGON ((-122.29273 47.72019, -122.29273 47.7..."
...,...,...
477,530330117003,"POLYGON ((-122.28084 47.52520, -122.28082 47.5..."
478,530330117004,"POLYGON ((-122.27932 47.52258, -122.27954 47.5..."
479,530330118001,"POLYGON ((-122.26232 47.53173, -122.24933 47.5..."
480,530330118002,"POLYGON ((-122.25464 47.51692, -122.25455 47.5..."


In [32]:
# merge the two dataframes
gdf= bg.merge(df, on='GEOID10')
gdf


,GEOID10,geometry,pop_density,median_income
0,530330001001,"POLYGON ((-122.26840 47.72641, -122.26719 47.7...",3708.992,206400.0
1,530330001002,"POLYGON ((-122.28970 47.73193, -122.28965 47.7...",14342.320,50714.0
2,530330001003,"POLYGON ((-122.28633 47.72093, -122.28629 47.7...",10101.800,51397.0
3,530330001004,"POLYGON ((-122.29237 47.73194, -122.29231 47.7...",51934.570,42150.0
4,530330001005,"POLYGON ((-122.29273 47.72019, -122.29273 47.7...",15089.690,47292.0
...,...,...,...,...
463,530330117002,"POLYGON ((-122.28750 47.52489, -122.28732 47.5...",2603.928,84886.0
464,530330117003,"POLYGON ((-122.28084 47.52520, -122.28082 47.5...",6885.910,91583.0
465,530330117004,"POLYGON ((-122.27932 47.52258, -122.27954 47.5...",2112.793,72639.0
466,530330118001,"POLYGON ((-122.26232 47.53173, -122.24933 47.5...",5266.039,105667.0


In [44]:
#visulize the data
gdf.explore(tiles = 'cartodbpositron', column ='median_income', cmap='GnBu_r', scheme='natural_breaks', k=10, legend=True)

In [56]:
#save gdf to shapefile, make directory if it does not exist
import os
if not os.path.exists('../data/shapefiles/seattle'):
    os.makedirs('../data/shapefiles/seattle')
gdf.to_file('../data/shapefiles/seattle/seattle.shp')

/var/folders/zm/x5dzdlrs5mlbxrm7z5g6p05r0000gn/T/ipykernel_22124/701296342.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file('../data/shapefiles/seattle/seattle.shp')


In [58]:
#connect to duckdb
con = duckdb.connect(database='../data/my_spatial_db.duckdb', read_only=False)

In [59]:
#install the spatial extension
con.install_extension("spatial")
con.load_extension("spatial")

In [60]:
#read the shapefile into duckdb
con.sql("SELECT * FROM ST_Read('../data/shapefiles/seattle/seattle.shp')")

┌──────────────┬───────────────────┬────────────┬──────────────────────────────────────────────────────────────────────┐
│   GEOID10    │    pop_densit     │ median_inc │                                 geom                                 │
│    int64     │      double       │   double   │                               geometry                               │
├──────────────┼───────────────────┼────────────┼──────────────────────────────────────────────────────────────────────┤
│ 530330001001 │          3708.992 │   206400.0 │ POLYGON ((-122.26839878436476 47.726405904524746, -122.26719075207…  │
│ 530330001002 │          14342.32 │    50714.0 │ POLYGON ((-122.28970480907557 47.73193185821239, -122.289654800872…  │
│ 530330001003 │           10101.8 │    51397.0 │ POLYGON ((-122.28632675942778 47.7209318679193, -122.2862897518206…  │
│ 530330001004 │          51934.57 │    42150.0 │ POLYGON ((-122.29237280801134 47.731942852365506, -122.29231180027…  │
│ 530330001005 │          15089.

In [62]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS seattle_pop_income AS 
        SELECT * FROM ST_Read('../data/shapefiles/seattle/seattle.shp')
        """
)

In [63]:
#create a new table
con.table('seattle_pop_income')

┌──────────────┬───────────────────┬────────────┬──────────────────────────────────────────────────────────────────────┐
│   GEOID10    │    pop_densit     │ median_inc │                                 geom                                 │
│    int64     │      double       │   double   │                               geometry                               │
├──────────────┼───────────────────┼────────────┼──────────────────────────────────────────────────────────────────────┤
│ 530330001001 │          3708.992 │   206400.0 │ POLYGON ((-122.26839878436476 47.726405904524746, -122.26719075207…  │
│ 530330001002 │          14342.32 │    50714.0 │ POLYGON ((-122.28970480907557 47.73193185821239, -122.289654800872…  │
│ 530330001003 │           10101.8 │    51397.0 │ POLYGON ((-122.28632675942778 47.7209318679193, -122.2862897518206…  │
│ 530330001004 │          51934.57 │    42150.0 │ POLYGON ((-122.29237280801134 47.731942852365506, -122.29231180027…  │
│ 530330001005 │          15089.

In [64]:
con.close()